In [40]:
import pandas as pd
import numpy as np
import re
from datasketch import MinHash, MinHashLSH

df = pd.read_csv('spotify_dataset/data.csv')
df

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.9950,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563000,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.9940,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901000,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.6040,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.9950,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887000,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.9900,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908000,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,0.1730,"['DripReport', 'Tyga']",0.875,163800,0.4430,1,4KppkflX7I3vJQk7urOJaS,0.000032,1,0.0891,-7.461,1,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012,0.3060,2020
169905,0.0167,"['Leon Bridges', 'Terrace Martin']",0.719,167468,0.3850,0,1ehhGlTvjtHo2e4xJFB0SZ,0.031300,8,0.1110,-10.907,1,Sweeter (feat. Terrace Martin),64,2020-06-08,0.0403,128.000,0.2700,2020
169906,0.5380,"['Kygo', 'Oh Wonder']",0.514,180700,0.5390,0,52eycxprLhK3lPcRLbQiVk,0.002330,7,0.1080,-9.332,1,How Would I Know,70,2020-05-29,0.1050,123.700,0.1530,2020
169907,0.0714,"['Cash Cash', 'Andy Grammer']",0.646,167308,0.7610,0,3wYOGJYD31sLRmBgCvWxa4,0.000000,1,0.2220,-2.557,1,I Found You,70,2020-02-28,0.0385,129.916,0.4720,2020


In [45]:
df = df[['id','acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness','speechiness', 'tempo', 'valence', 'year']]

In [46]:
df2= df[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness','speechiness', 'tempo', 'valence', 'year']]

In [47]:
def create_minhash_signatures(df, lsh_threshold, num_perm):
    min_hashes = []
    for i in range(len(df)):
        row = df.iloc[i]
        m = MinHash(num_perm = num_perm)
        for s in row:
            m.update(str(s).encode('utf8'))
        min_hashes.append(m)
        
    lsh = MinHashLSH(lsh_threshold, num_perm) 
    for key, minhash in enumerate(min_hashes):
        lsh.insert(key, minhash)
    return(min_hashes, lsh)

minhashes, lsh_signatures = create_minhash_signatures(df2, lsh_threshold = 0.5, num_perm = 6)

In [48]:
def query_lsh(minhashes, lsh_signature):
    results = []
    for idx, minhash in enumerate(minhashes):
        result = lsh_signature.query(minhash)
        results.append(result)
    return(results)
results = query_lsh(minhashes, lsh_signatures)

In [52]:
results[100]

[32256,
 153280,
 107683,
 100,
 78987,
 16268,
 175,
 147,
 135893,
 126615,
 16248,
 126649,
 135900,
 24318]

In [56]:
def display_results(results):
    for idx, res in enumerate(results[0:5]):
        print(idx,res)
        temp_lst = []
        for i in res:
            temp_lst.append(df2.iloc[i])
        print(df2.iloc[idx], "--->", temp_lst)

display_results(results)

0 [0]
acousticness             0.9950
danceability             0.7080
duration_ms         158648.0000
energy                   0.1950
instrumentalness         0.5630
liveness                 0.1510
loudness               -12.4280
speechiness              0.0506
tempo                  118.4690
valence                  0.7790
year                  1928.0000
Name: 0, dtype: float64 ---> [acousticness             0.9950
danceability             0.7080
duration_ms         158648.0000
energy                   0.1950
instrumentalness         0.5630
liveness                 0.1510
loudness               -12.4280
speechiness              0.0506
tempo                  118.4690
valence                  0.7790
year                  1928.0000
Name: 0, dtype: float64]
1 [1, 126415]
acousticness             0.9940
danceability             0.3790
duration_ms         282133.0000
energy                   0.0135
instrumentalness         0.9010
liveness                 0.0763
loudness               -28.45

In [60]:
min_hashes = []
m = MinHash(num_perm = 6)
for s in [0.9950,0.7080,158648.0000,0.1950,0.5630,0.1510,-12.4280,0.0506,118.4690,0.7790,1928.0000]:
    m.update(str(s).encode('utf8'))
min_hashes.append(m)
results2 = query_lsh(min_hashes, lsh_signatures)

In [62]:
results2

[[0]]

In [63]:
lsh_signatures